## Retrieval-Augmented Generation (RAG) as a structured pipeline. 

The goal is to move from "guessing" to "synthesizing verified information" by making retrieval a natural transformation step in your code.

In LCEL, this is achieved by transforming the user's raw input into a search query, fetching documents, and then piping those documents into a final prompt.

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. THE BRAIN: Our LLM
llm = ChatOpenAI(model="gpt-4o")

# 2. THE DATA SOURCE: A mock retriever representing "Verified Information"
# In a real app, this would be a Vector Database (Pinecone, Chroma, etc.)
def mock_vector_db(query: str):
    # This represents 'Truth becoming a structural outcome'
    verified_context = (
        "Solid-state batteries use solid electrolytes instead of liquid ones. "
        "They offer higher energy density and improved safety over lithium-ion."
    )
    return verified_context

# 3. THE BLUEPRINT: A prompt that forces the LLM to use the context
rag_prompt = ChatPromptTemplate.from_template("""
Answer the question based ONLY on the following context:
{context}

Question: {question}
""")

# 4. THE PIPELINE: LCEL integrates retrieval as a natural transformation
rag_chain = (
    # Step 1: Prepare the inputs
    # We pass the question through, and use the retriever to fetch context
    {"context": mock_vector_db, "question": RunnablePassthrough()}
    # Step 2: Feed into the prompt
    | rag_prompt 
    # Step 3: Synthesis by the LLM using verified info
    | llm 
    # Step 4: Clean string output
    | StrOutputParser()
)

# 5. EXECUTION
response = rag_chain.invoke("Why are solid-state batteries better?")
print(response)

Solid-state batteries are considered better because they use solid electrolytes instead of liquid ones, which offer higher energy density and improved safety over traditional lithium-ion batteries.
